In [0]:
#Notes
'''all the actions are case sensitive please be carefull while writing .'''

'all the actions are case sensitive please be carefull while writing .'

In [0]:
from pyspark import SparkContext

In [0]:
sc = SparkContext.getOrCreate()
print(sc)

<SparkContext master=local[8] appName=Databricks Shell>

In [0]:
my_rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10,11,11,12,13,14,15])

In [0]:
my_rdd.collect()


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11, 12, 13, 14, 15]

In [0]:
print(my_rdd.take(3))#display first 3 elements
print(my_rdd.top(3)) #display last 3 elements
print(my_rdd.min())#display minimum
print(my_rdd.max())#display maximum
print(my_rdd.sum())#doing sum of all
print(my_rdd.mean())#mean
print(my_rdd.stdev())#standard deviation


[1, 2, 3]

[15, 14, 13]

1

15

131

8.1875
4.245861956069698

In [0]:
#take a subset of list from list/want few samples
print(my_rdd.takeSample(True,9)) 
print(my_rdd.takeSample(False,9)) 

#Repeated Values:
#true = give some repeated values if have
#false = give unique values

[2, 8, 14, 9, 1, 13, 15, 7, 11]

[5, 15, 11, 10, 14, 6, 12, 4, 11]

In [0]:
#--------------------------------------------#

In [0]:
#reduce is a spark action that aggregates a dataset element
rdd = sc.parallelize([0,1,2,3,4,5,6,7,8,9])
rdd.reduce(lambda a,b:a+b)
  

Out[1]: 45

In [0]:
#COUNT THE ELEMENT
x = sc.parallelize([1,1,1,2,1,4,3,4,4,6,3,6,7,8,6,])
x.count()

15

In [0]:
#COUNT THE VALUE
x = sc.parallelize([1,1,1,2,1,4,3,4,4,6,3,6,7,8,6,])
x.countByValue()

defaultdict(int, {1: 4, 2: 1, 4: 3, 3: 2, 6: 3, 7: 1, 8: 1})

In [0]:

#COUNT by keys
x = sc.parallelize([('A',1),('B',2),('C',5),('C',6)])

x.countByKey()

defaultdict(int, {'A': 1, 'B': 1, 'C': 2})

In [0]:
'''Fold is a very powerful operation in spark which allows you to calculate many important values in O(n) time. 

def fold[T](acc:T)((acc,value) => acc)
The above is kind of high level view of fold api. It has following three things

->T is the data type of RDD
->acc is accumulator of type T which will be return value of the fold operation
->A function , which will be called for each element in rdd with previous accumulator.'''
#aggregate the elements of each partition
from operator import add

x_fold = sc.parallelize([1,2,3,4,5])
D=x_fold.repartition(1)
D.fold(0,add)

15

In [0]:
print(x_fold.getNumPartitions())

8

In [0]:
x_fold.glom().collect()
#go through " https://www.youtube.com/watch?v=h9ns7DHS6fI "

[[], [1], [], [2], [3], [], [4], [5]]

In [0]:
#Range 
x_range = sc.parallelize(range(0,10))#or(range(20))
x_range.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [0]:
#variance
x_variance = sc.parallelize([1,2,3,4,5,6,7])
x_variance.variance()

4.0

In [0]:
'''
difference between variance and sample variance is variance can take n value but sample variance can take n-1 values as an input
'''

In [0]:
#samplevariance
x_samplevar = sc.parallelize([1,2,3,4,5,6])
x_samplevar.sampleVariance()

3.5

In [0]:
#save as picklefile
nRdd= sc.textFile("/FileStore/tables/data-1.txt")
nRdd.collect()

In [0]:
newvar = sc.parallelize([1,2,3,4,5])

In [0]:
newvar.saveAsTextFile("DBFS/FileStore/rdddata1/") 

---------------------------------------------------------------------------
Py4JJavaError Traceback (most recent call last)
 in 
----> 1 newvar.saveAsTextFile("DBFS/FileStore/rdddata1/")

/databricks/spark/python/pyspark/rdd.py in saveAsTextFile(self, path, compressionCodecClass)
 1861 self.ctx._jvm.PythonRDD.saveAsTextFileImpl(keyed._jrdd, path, compressionCodecClass)
 1862 else:
-> 1863 self.ctx._jvm.PythonRDD.saveAsTextFileImpl(keyed._jrdd, path)
 1864 
 1865 # Pair functions

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__(self, *args)
 1302 
 1303 answer = self.gateway_client.send_command(command)
-> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
 115 def deco(*a, **kw):
 116 try:
--> 117 return f(*a, **kw)
 118 except py4j.protocol.Py4JJavaError as e:
 119 converted = convert_exception(e.java_exception)

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value(answer, gateway_client, target_id, name)
 324 value = OUTPUT_CONVERTER[type](answer[2:], gateway_client)
 325 if answer[1] == REFERENCE_TYPE:
--> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n".
 328 format(target_id, ".", name), value)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.saveAsTextFileImpl.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory dbfs:/DBFS/FileStore/rdddata1 already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:302)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:75)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1090)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:125)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:419)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1088)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1061)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:125)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:419)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopFile(PairRDDFunctions.scala:1026)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$3(PairRDDFunctions.scala:1008)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:125)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:419)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopFile(PairRDDFunctions.scala:1007)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$2(PairRDDFunctions.scala:964)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:125)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD

In [0]:
newvar.collect()

[1, 2, 3, 4, 5]

In [0]:

rdd_emp.coalesce(1).saveAsTextFile("/FileStore/tables/data-1.txt")

---------------------------------------------------------------------------
NameError Traceback (most recent call last)
 in 
----> 1 rdd_emp.coalesce(1).saveAsTextFile("/FileStore/tables/data-1.txt")

NameError: name 'rdd_emp' is not defined

In [0]:
#transformation 
'''
spark transformation is a function that produces new RDD from the existing RDDs. It takes RDD as input and produces one or mote RDD as output. Each time it creates new RDD when we apply any transformation.

OR:->lazy execution is there:delay execution untill finds an action so that it can prepare optimized lineage....

go through:  "  https://drive.google.com/drive/folders/1jR8bg4V2_24kquRpthsFTmDuWQmK1UwJ "
'''

'\nspark transformation is a function that produces new RDD from the existing RDDs. It takes RDD as input and produces one or mote RDD as output. Each time it creates new RDD when we apply any transformation.\n\nOR:->lazy execution is there:delay execution untill finds an action so that it can prepare optimized lineage....\n\ngo through: " https://drive.google.com/drive/folders/1jR8bg4V2_24kquRpthsFTmDuWQmK1UwJ "\n'

In [0]:
#narrow:map():map funtion iterates over every line in RDD and split into new RDD

x_map = sc.parallelize([1,2,3,4,5,6])
y_map = x_map.map(lambda x:(x+2))#eg-(x,x+2)....etc
y_map.collect()

Out[15]: [3, 4, 5, 6, 7, 8]

In [0]:
#narrow:filter():returns a new RDD, containing only the elements that meet a predicate.
#narrow operation because it does not shuffle data from one partition to many partition

x_filter = sc.parallelize([1,2,3,4,5,6,7,8,9])
y_filter = x_filter.filter(lambda x:x%2==0)
y_filter.collect()

[2, 4, 6, 8]

In [0]:
#narrow:flatMap():in this each input have many elements in an output RDD.or Split each input string into words.
#multiple or secuence value as we do not do this in map() because this is for single output/input


x_flatMap = sc.parallelize([1,2,3,4,5,6,7,8,9])
y_flatMap = x_flatMap.flatMap(lambda x:(x,x**2))#like use for multiple works
y_flatMap.collect()


[1, 1, 2, 4, 3, 9, 4, 16, 5, 25, 6, 36, 7, 49, 8, 64, 9, 81]

In [0]:
#mapPartitions():similar to map,but runs separately on each partition of the RDD
"""
 converts each partition of the source RDD into many elements of the result (possibly none). In mapPartition(), the map() function is applied on each partitions simultaneously. MapPartition is like a map, but the difference is it runs separately on each partition(block) of the RDD.
"""
x = sc.parallelize([1,2,3,4,5,6,7,8,9,],3 )
def f(iter):
  yield sum(iter)
y=x.mapPartitions(f)
y.collect()

[6, 15, 24]

In [0]:
#glom():flattens elements on the same partitions
print(x.glom().collect())

print(y.glom().collect())

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
[[6], [15], [24]]

In [0]:
#mapPartitionsWithIndex():
'''
 mapPartition it provides func with an integer value representing the index of the partition, and the map() is applied on partition index wise one after the other.
 
 '''
#here we can see the partitions including the sum.....
x = sc.parallelize([1,2,3,4,5,6,7,8,9,],2 )
def f(pos,iter):
  yield (pos,sum(iter))
y=x.mapPartitionsWithIndex(f)
y.collect()


[(0, 10), (1, 35)]

In [0]:
#sample():Sample a fraction fraction of the data, with or without replacement, using a given random number generator seed.
#sample(withReplacement:True/False,fraction,seed=None/optional)
x = sc.parallelize([1,2,3,4,5,6,7,8,9])
y = x.sample(True,0.3)
y.collect()

[6]

In [0]:
#union
x = sc.parallelize([3,2,4,7])
y = sc.parallelize([3,4,6,9,0])
x.union(y).collect()



[3, 2, 4, 7, 3, 4, 6, 9, 0]

In [0]:
#intersection
x = sc.parallelize([3,2,4,7])
y = sc.parallelize([3,4,6,9,0])
x.intersection(y).collect()

[3, 4]

In [0]:
#distinct:duplicates is removed
x = sc.parallelize([3,3,3,2,4,7])

x.distinct().collect()

[2, 3, 4, 7]

In [0]:
#groupByKey()
x = sc.parallelize([('A',3),('A',6),('B',2),('C',9)])
y=x.groupByKey()
y.collect()

In [0]:
#Please don't use this because because this is closure problem we know that we initiate the spark context or the driver code and initialise our counter as zero and after that create an RDD out of that and execute our program.What happens is like drivers submit the program to the worker node because we have three or may be more worker node and they having individual executor in each worker node.driver initialize a serialized copy of counter =0 to each of working node to use it for a particular task. But actually all the executors play with the counter not do anything with it. Thats why they return counter as 0.

#like in laymen language Datanode holds the RDD part and the increment of counter is made by driver program so when the counter is initialize to 0 they save it to 0.
#To solve this closure problem, Spark introduce a concept called accumulator, it will aggregate values from worker back to driver. Only driver can access the value of a accumulator. For task, accumulator are write only.
counter = 0

rdd = sc.parallelize([1,2,3,4,5,6,7,8])

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x
rdd.foreach(increment_counter)

print("Counter value: ", counter)


Counter value: 0